<a href="https://colab.research.google.com/github/senura96/HuggingFace-Tranformers/blob/main/Emotion_Prediction_Using_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**bold text**

# **Emotion Prediction using HuggingFace Transformets**



In [1]:
#Installing Required Libraries
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets
!pip install -U bertviz
!pip install -U umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 20.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.7 MB/s eta 0:00:00
     ━━━

In [36]:
!pip install datasets evaluate transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.0 MB/s eta 0:00:00


In [8]:
import pandas as pd
from datasets import list_datasets , load_dataset

In [6]:
all_datasets = list_datasets()
len(all_datasets)

92208

In [9]:
#Loading Emotion Dataset

emotion = load_dataset('emotion')


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [10]:
print(emotion)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


In [59]:
import torch
from transformers import AutoTokenizer , DataCollatorWithPadding , AutoModel , AutoModelForSequenceClassification,TrainingArguments , Trainer
import numpy as np

In [18]:

# Importing Tokeniz
model_checkpoint = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [15]:
# Tokenization Function
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True)

In [20]:
emotions_encoded = emotion.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

In [22]:
print(emotions_encoded["train"][1])

{'text': 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake', 'label': 0, 'input_ids': [101, 1045, 2064, 2175, 2013, 3110, 2061, 20625, 2000, 2061, 9636, 17772, 2074, 2013, 2108, 2105, 2619, 2040, 14977, 1998, 2003, 8300, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [26]:
#Loading the Model
model = AutoModel.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels =6).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
device

device(type='cuda')

In [54]:

batch_size = 64
model_name = "distilbert-finetuned-emotion"

batch_size = 32  # Smaller batches may allow for better generalization
model_name = "distilbert-finetuned-emotion"

training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=10,
    learning_rate=5e-05,  # Slightly lower learning rate for better convergence
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    disable_tqdm=False,
    warmup_steps=200,  # Fewer warm-up steps for quicker convergence
    gradient_accumulation_steps=2,  # Accumulate gradients over 2 steps for larger effective batch size
    lr_scheduler_type='cosine',
    fp16=True,
    logging_dir='./logs',
    save_total_limit=3,
    metric_for_best_model='eval_accuracy',  # Use accuracy as the metric for best model selection
    save_steps=500,  # Save more frequently
    eval_steps=250,  # Evaluate more frequently
    # Add any additional model configuration parameters here
)


In [38]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average='weighted')
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}


In [55]:
trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded['train'],
                  eval_dataset=emotions_encoded['validation'],
                  tokenizer=tokenizer
                  )


In [56]:

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.356996,0.929500,0.929774
2,0.023400,0.365647,0.938500,0.938380
3,0.023400,0.379456,0.938500,0.938349
4,0.014300,0.351786,0.931000,0.930669
5,0.014300,0.428827,0.937500,0.937404
6,0.006900,0.450261,0.935000,0.935119
7,0.006900,0.452360,0.932500,0.932538
8,0.004200,0.455090,0.935500,0.935572
9,0.004200,0.452155,0.935000,0.935080
10,0.003200,0.453020,0.935500,0.935560


Checkpoint destination directory distilbert-finetuned-emotion/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=2500, training_loss=0.010397321844100952, metrics={'train_runtime': 303.8543, 'train_samples_per_second': 526.568, 'train_steps_per_second': 8.228, 'total_flos': 2158586043200640.0, 'train_loss': 0.010397321844100952, 'epoch': 10.0})

In [57]:
predictions = trainer.predict(emotions_encoded['test']
                               )


print(predictions)

PredictionOutput(predictions=array([[ 9.1484375, -3.4121094, -5.6015625, -2.7539062, -2.6601562,
        -4.125    ],
       [ 9.2578125, -3.7832031, -5.890625 , -2.2324219, -2.6171875,
        -4.2539062],
       [ 9.1953125, -3.9042969, -5.7265625, -2.6679688, -2.3574219,
        -4.0351562],
       ...,
       [-4.3164062,  8.9296875, -3.109375 , -5.2890625, -4.765625 ,
        -3.65625  ],
       [-4.4023438,  9.0625   , -2.3828125, -5.7304688, -4.921875 ,
        -4.2773438],
       [-5.0703125, -5.2304688, -7.28125  , -4.1210938,  7.21875  ,
         1.8173828]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 1, 4]), metrics={'test_loss': 0.5073036551475525, 'test_accuracy': 0.9285, 'test_f1': 0.9282469515198645, 'test_runtime': 1.0144, 'test_samples_per_second': 1971.683, 'test_steps_per_second': 62.108})


In [71]:
preds = np.argmax(predictions.predictions, axis=-1)
preds_emotions = ['Saddness' if preds == 0 else 'Joy' if preds == 1 else 'Love' if preds == 2 else 'Anger' if preds == 3 else 'Fear' if preds == 4 else 'Surprise' if preds == 5 else 'Unknown'
                  for preds in preds]

print(preds_emotions)


['Saddness', 'Saddness', 'Saddness', 'Joy', 'Saddness', 'Fear', 'Fear', 'Joy', 'Joy', 'Anger', 'Anger', 'Saddness', 'Fear', 'Joy', 'Love', 'Saddness', 'Joy', 'Saddness', 'Anger', 'Joy', 'Saddness', 'Joy', 'Joy', 'Saddness', 'Saddness', 'Fear', 'Anger', 'Saddness', 'Fear', 'Anger', 'Fear', 'Anger', 'Saddness', 'Anger', 'Saddness', 'Joy', 'Joy', 'Saddness', 'Joy', 'Joy', 'Saddness', 'Saddness', 'Joy', 'Saddness', 'Joy', 'Anger', 'Joy', 'Joy', 'Fear', 'Fear', 'Saddness', 'Fear', 'Joy', 'Saddness', 'Joy', 'Saddness', 'Saddness', 'Joy', 'Saddness', 'Anger', 'Saddness', 'Saddness', 'Joy', 'Joy', 'Saddness', 'Surprise', 'Saddness', 'Anger', 'Fear', 'Surprise', 'Joy', 'Love', 'Fear', 'Joy', 'Love', 'Anger', 'Joy', 'Saddness', 'Joy', 'Love', 'Joy', 'Anger', 'Saddness', 'Joy', 'Saddness', 'Saddness', 'Love', 'Joy', 'Joy', 'Saddness', 'Joy', 'Fear', 'Anger', 'Fear', 'Anger', 'Anger', 'Joy', 'Saddness', 'Fear', 'Saddness', 'Saddness', 'Saddness', 'Saddness', 'Anger', 'Anger', 'Anger', 'Joy', 'Joy'

In [76]:
for i in zip(emotion['test']['text'] , preds_emotions):

    print(i)



df = pd.DataFrame()
df["text"] = emotion['test']['text']
df["emotion" ]= preds_emotions
print(df)

('im feeling rather rotten so im not very ambitious right now', 'Saddness')
('im updating my blog because i feel shitty', 'Saddness')
('i never make her separate from me because i don t ever want her to feel like i m ashamed with her', 'Saddness')
('i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived', 'Joy')
('i was feeling a little vain when i did this one', 'Saddness')
('i cant walk into a shop anywhere where i do not feel uncomfortable', 'Fear')
('i felt anger when at the end of a telephone call', 'Fear')
('i explain why i clung to a relationship with a boy who was in many ways immature and uncommitted despite the excitement i should have been feeling for getting accepted into the masters program at the university of virginia', 'Joy')
('i like to have the same breathless feeling as a reader eager to see what will happen next', 'Joy')
('i jest i feel grumpy tired and pre menstrual which i probably am but then again its on

In [77]:
df

,text,emotion
0,im feeling rather rotten so im not very ambiti...,Saddness
1,im updating my blog because i feel shitty,Saddness
2,i never make her separate from me because i do...,Saddness
3,i left with my bouquet of red and yellow tulip...,Joy
4,i was feeling a little vain when i did this one,Saddness
...,...,...
1995,i just keep feeling like someone is being unki...,Anger
1996,im feeling a little cranky negative after this...,Anger
1997,i feel that i am useful to my people and that ...,Joy
1998,im feeling more comfortable with derby i feel ...,Joy
